In [1]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import cv2

#from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Conv2D,Flatten,MaxPooling2D,AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.applications.vgg16 import VGG16


# import keras_tuner as kt


2024-10-19 22:45:15.856264: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 22:45:15.860079: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-19 22:45:15.951512: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-19 22:45:15.999438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 22:45:16.088427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
folder_path="/home/aman/Desktop/Python/ML/Datasets/data/"

In [3]:
img_path = []
for file in os.listdir(folder_path+"with_mask"):
  img_path.append(file)

len(img_path)
df=pd.DataFrame({'image':img_path,'mask':1})

In [4]:
df.head()

,image,mask
0,with_mask_10.jpg,1
1,with_mask_100.jpg,1
2,with_mask_1001.jpg,1
3,with_mask_1003.jpg,1
4,with_mask_1008.jpg,1


In [5]:
new_img= []
for file in os.listdir(folder_path+"without_mask"):
  new_img.append(file)
new_df=pd.DataFrame({'image':new_img,'mask':0})

In [6]:
df.shape

(2500, 2)

In [7]:
new_df.shape

(2500, 2)

In [8]:
result = pd.concat([df,new_df],ignore_index=True)

In [9]:
result

,image,mask
0,with_mask_10.jpg,1
1,with_mask_100.jpg,1
2,with_mask_1001.jpg,1
3,with_mask_1003.jpg,1
4,with_mask_1008.jpg,1
...,...,...
4995,without_mask_2698.jpg,0
4996,without_mask_1610.jpg,0
4997,without_mask_3349.jpg,0
4998,without_mask_3195.jpg,0


In [10]:
final_df= result.sample(frac=1, random_state=1).reset_index(drop=True)
final_df

,image,mask
0,without_mask_1548.jpg,0
1,without_mask_2685.jpg,0
2,without_mask_3647.jpg,0
3,without_mask_30.jpg,0
4,without_mask_1473.jpg,0
...,...,...
4995,without_mask_1816.jpg,0
4996,without_mask_1547.jpg,0
4997,with_mask_3262.jpg,1
4998,without_mask_572.jpg,0


In [11]:
data =[]
label = []
for i in range(final_df.shape[0]):
  name,mask= (final_df.iloc[i,:]).tolist()
  if (mask == 1):
    img_path=os.path.join(folder_path+"with_mask/",name)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    # img=np.array(img)
    # label.append(1)
    data.append([img,mask])

    # data.append(img)
  else :
    img_path = os.path.join(folder_path+'without_mask/',name)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    # img=np.array(img)
    # label.append(0)
    data.append([img,mask])

    # data.append(img)



In [12]:
# X=np.array(data)
# y=np.array(label)
# X=X/255

X=[]
y=[]
for features,label in data:
    X.append(features)
    y.append(label)
    
X=np.array(X)
y=np.array(y)
X=X/255

In [13]:
X.shape

(5000, 224, 224, 3)

In [14]:
y.shape

(5000,)

In [20]:
shuffler = lambda x: np.random.permutation(x)

In [15]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
X_train.shape

(4000, 224, 224, 3)

In [17]:
X_test.shape

(1000, 224, 224, 3)

In [21]:
X=5
y=2
data=[]
label=[]
result=[]
df=[]
new_df=[]
final_df=[]
image_path=[]
img=[]
new_img=[]

In [22]:
conv_base = VGG16(input_shape=(224,224,3))

model = Sequential()
for layer in conv_base.layers[:-1]:
    layer.trainable = False
    model.add(layer)


model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,264,641 (512.18 MB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 134,260,544 (512.16 MB)

In [23]:
model.compile(optimizer='Adam',metrics=['accuracy'],loss='binary_crossentropy')

In [24]:
history = model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test),batch_size=32)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 601s 5s/step - accuracy: 0.6781 - loss: 0.5777 - val_accuracy: 0.8150 - val_loss: 0.3891
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 552s 4s/step - accuracy: 0.8752 - loss: 0.3216 - val_accuracy: 0.9110 - val_loss: 0.2635
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 533s 4s/step - accuracy: 0.9102 - loss: 0.2547 - val_accuracy: 0.9090 - val_loss: 0.2516
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 514s 4s/step - accuracy: 0.9176 - loss: 0.2242 - val_accuracy: 0.9380 - val_loss: 0.2036
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 542s 4s/step - accuracy: 0.9148 - loss: 0.2240 - val_accuracy: 0.9400 - val_loss: 0.1876
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 538s 4s/step - accuracy: 0.9369 - loss: 0.1826 - val_accuracy: 0.9380 - val_loss: 0.1952
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 535s 4s/step - accuracy: 0.9347 - loss: 0.1795 - val_accuracy: 0.9460 - val_loss: 0.1656
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 520s 4s/step - accuracy: 0.9412 - loss: 0.1564 - val_accu

In [25]:
model.save('final_face.keras')

In [28]:
model.save("face_mask.keras")

In [ ]:
cap= cv2.VideoCapture(0)

while True:
    ret ,frame = cap.read()
    # call the detection 
    frame =cv2.resize(frame,(150,150))
    y_pred= detectface_mask(frame)
    cv2.imshow("window",frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
cv2.destroyAllWindows()

def detectface_mask(img):
    y_pred = model.predict(img.reshape(1,150,150,3))
    return y_pred[0][0]

sample_1=cv2.imread('/home/aman/Desktop/Python/ML/Mask/mask.png')
sample_1 = cv2.resize(sample_1,(150,150))

detectface_mask(sample_1)

In [41]:
final_df

,image,mask
0,with_mask_3172.jpg,1
1,with_mask_2976.jpg,1
2,without_mask_28.jpg,0
3,with_mask_2166.jpg,1
4,without_mask_372.jpg,0
...,...,...
7548,with_mask_1813.jpg,1
7549,without_mask_2319.jpg,0
7550,without_mask_1228.jpg,0
7551,with_mask_121.jpg,1
